In [1]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd
import joblib
import numpy as np
from collections import defaultdict
import pandas as pd
# import story_helper
import sasc.config
from sasc.modules.fmri_module import convert_module_num_to_voxel_num, add_stability_score
from sasc.config import FMRI_DIR, STORIES_DIR

# Read all the info from stories into a single pickle file

In [2]:
# load stuff
# double check all of these, intro paragraph may be the same...
output_file = join(sasc.config.RESULTS_DIR,
                   'processed', "pilot3_story_data.pkl")
story_mapping = {
    'default/uts03___jun14___seed=5': 'GenStory12_resps.npy',
    'default/uts03___jun14___seed=1': 'GenStory13_resps.npy',

    'interactions/uts03___jun14___seed=5': 'GenStory14_resps.npy',
    'interactions/uts03___jun14___seed=6': 'GenStory15_resps.npy',

    'polysemantic/uts03___jun14___seed=3': 'GenStory16_resps.npy',
    'polysemantic/uts03___jun14___seed=7': 'GenStory17_resps.npy',
}

In [3]:
# cluster_neighbors = joblib.load(join(FMRI_DIR, "voxel_neighbors_and_pcs", "cluster_neighbors_v1.pkl"))
perfs = joblib.load(join(sasc.config.FMRI_DIR, 'sasc', 'rj_models',
                    'opt_model', 'new_setup_performance.jbl'))

# add keys
stories_data_dict = defaultdict(list)
for story_idx, story_name in enumerate(story_mapping.keys()):
    # add scalar story descriptions
    stories_data_dict["story_name_original"].append(story_name)
    stories_data_dict["story_setting"].append(story_name.split("/")[0])
    stories_data_dict["story_name_new"].append(story_mapping[story_name])
    stories_data_dict["story_text"].append(
        open(join(STORIES_DIR, story_name, "story.txt"), "r").read()
    )
    prompts_paragraphs = joblib.load(
        join(STORIES_DIR, story_name, "prompts_paragraphs.pkl")
    )

    # add paragraph-level descriptions
    timings = pd.read_csv(
        join(STORIES_DIR, story_name, "timings_processed.csv"))
    # timings['time_running']
    stories_data_dict["timing"].append(timings)
    stories_data_dict["prompts"].append(prompts_paragraphs["prompts"])
    stories_data_dict["paragraphs"].append(prompts_paragraphs["paragraphs"])

    # add paragraph-level metadata
    # rows
    # rows = pd.read_csv(join(STORIES_DIR, story_name, "rows.csv"))
    story_metadata_per_paragraph = pd.read_pickle(
        join(STORIES_DIR, story_name, "rows.pkl"))
    story_metadata_per_paragraph["voxel_num"] = story_metadata_per_paragraph.apply(
        lambda row: convert_module_num_to_voxel_num(
            row["module_num"], row["subject"]),
        axis=1,
    )
    story_metadata_per_paragraph["stability_score"] = story_metadata_per_paragraph.apply(
        lambda row: add_stability_score(row["module_num"], row["subject"]),
        axis=1,
    )
    story_metadata_per_paragraph = story_metadata_per_paragraph[
        [
            "expl",
            "module_num",
            "top_explanation_init_strs",
            "subject",
            "fmri_test_corr",
            "stability_score",
            # "top_score_synthetic",
            "top_score_normalized",
            "roi_anat",
            "roi_func",
            "voxel_num",
        ]
    ]
    story_metadata_per_paragraph['test_corr_new'] = story_metadata_per_paragraph['voxel_num'].apply(
        lambda x: perfs[x])
    # rows['cluster_nums'] = rows['voxel_num'].map(cluster_neighbors)
    stories_data_dict["rows"].append(story_metadata_per_paragraph)

    if "interactions" in list(story_mapping.keys())[story_idx]:
        rows1 = pd.read_pickle(join(STORIES_DIR, story_name, "rows1.pkl"))
        rows2 = pd.read_pickle(join(STORIES_DIR, story_name, "rows2.pkl"))
        rows1["voxel_num"] = rows1.apply(
            lambda row: convert_module_num_to_voxel_num(
                row["module_num"], row["subject"]
            ),
            axis=1,
        )
        rows2["voxel_num"] = rows2.apply(
            lambda row: convert_module_num_to_voxel_num(
                row["module_num"], row["subject"]
            ),
            axis=1,
        )
        stories_data_dict['voxel_num1'].append(rows1['voxel_num'])
        stories_data_dict['voxel_num2'].append(rows2['voxel_num'])
        stories_data_dict['expl1'].append(rows1['expl'])
        stories_data_dict['expl2'].append(rows2['expl'])
    else:
        stories_data_dict['voxel_num1'].append([])
        stories_data_dict['voxel_num2'].append([])
        stories_data_dict['expl1'].append([])
        stories_data_dict['expl2'].append([])


joblib.dump(stories_data_dict, output_file)

['/home/chansingh/automated-explanations/results/processed/pilot3_story_data.pkl']

In [4]:
output_file = join(sasc.config.RESULTS_DIR,
                   'processed', "pilot3_story_data.pkl")
stories_data_dict = joblib.load(output_file)
stories_data_dict['rows'][0].to_csv('uts03_default_voxels.csv')